# TensorFlow实现进阶的卷积神经网络

In [2]:
import cifar10 # 将下载的TensorFlow Models库里的cifar10放到项目中
import cifar10_input # 将cifar文件夹中的cifar10_input.py复制到外层文件夹中
import tensorflow as tf
import numpy as np
import time

In [3]:
# 定义训练轮数max_steps
max_steps = 3000
# 定义batch_size
batch_size = 128
# 定义下载CIFAR-10数据的路径
data_dir = 'temp/cifar10_data/cifar-10-batches-bin'

In [4]:
def variable_with_weight_loss(shape, stddev, wl):
    var = tf.Variable(tf.truncated_normal(shape, stddev=stddev))
    if wl is not None:
        weight_loss = tf.multiply(tf.nn.l2_loss(var), wl, name='weight_loss')
        tf.add_to_collection('losses', weight_loss)
    return var

In [5]:
# cifar10.maybe_download_and_extract() # 'module' object has no attribute 'maybe_download_and_extract'
# download_url: http://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz
# 准备训练数据
images_train, labels_train = cifar10_input.distorted_inputs(data_dir=data_dir, batch_size=batch_size)
# 准备测试数据
images_test, labels_test = cifar10_input.inputs(eval_data=True, data_dir=data_dir, batch_size=batch_size)

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


In [6]:
# 创建输入数据的placeholder
image_holder = tf.placeholder(tf.float32, [batch_size, 24, 24, 3])
label_holder = tf.placeholder(tf.int32, [batch_size])

In [7]:
# 创建第一个卷积层
weight1 = variable_with_weight_loss(shape=[5, 5, 3, 64], stddev=5e-2, wl=0.0)
kernel1 = tf.nn.conv2d(image_holder, weight1, [1, 1, 1, 1], padding='SAME')
bias1 = tf.Variable(tf.constant(0.0, shape=[64]))
conv1 = tf.nn.relu(tf.nn.bias_add(kernel1, bias1))
pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)

In [8]:
# 创建第二个卷积层
weight2 = variable_with_weight_loss(shape=[5, 5, 64, 64], stddev=5e-2, wl=0.0)
kernel2 = tf.nn.conv2d(norm1, weight2, [1, 1, 1, 1], padding='SAME')
bias2 = tf.Variable(tf.constant(0.1, shape=[64]))
conv2 = tf.nn.relu(tf.nn.bias_add(kernel2, bias2))
norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75)
pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

In [9]:
# 创建全连接层
reshape = tf.reshape(pool2, [batch_size, -1])
dim = reshape.get_shape()[1].value
weight3 = variable_with_weight_loss(shape=[dim, 384], stddev=0.04, wl=0.004)
bias3 = tf.Variable(tf.constant(0.1, shape=[384]))
local3 = tf.nn.relu(tf.matmul(reshape, weight3) + bias3)

In [10]:
# 类似全连接层，隐含节点数下降一半
weight4 = variable_with_weight_loss(shape=[384, 192], stddev=0.04, wl=0.004)
bias4 = tf.Variable(tf.constant(0.1, shape=[192]))
local4 = tf.nn.relu(tf.matmul(local3, weight4) + bias4)

In [11]:
# 最后一层
weight5 = variable_with_weight_loss(shape=[192, 10], stddev=1/192, wl=0.0)
bias5 = tf.Variable(tf.constant(0.0, shape=[10]))
logits = tf.add(tf.matmul(local4, weight5), bias5)

In [12]:
def loss(logits, labels):
    labels = tf.cast(labels, tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels, name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
    tf.add_to_collection('losses', cross_entropy_mean)
    return tf.add_n(tf.get_collection('losses'), name='total_loss')

In [13]:
loss = loss(logits, label_holder)

In [14]:
# 选择Adam优化器，设定学习速率为1e-3
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [15]:
# 输出Top k
top_k_op = tf.nn.in_top_k(logits, label_holder, 1)

In [16]:
# 创建session
session = tf.InteractiveSession()
# 变量全局初始化

tf.global_variables_initializer().run()

In [17]:
# 启动图片数据增强线程队列
tf.train.start_queue_runners()

[<Thread(QueueRunnerThread-input_producer-input_producer/input_producer_EnqueueMany, started daemon 123145341480960)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 123145345687552)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 123145349894144)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 123145354100736)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 123145358307328)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 123145362513920)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 123145366720512)>,
 <Thread(QueueRunnerThread-shuffle_batch/

In [18]:
# 正式开始训练
for step in range(max_steps):
    start_time = time.time()
    image_batch, label_batch = session.run([images_train, labels_train])
    _, loss_value = session.run([train_op, loss], feed_dict={image_holder: image_batch, label_holder: label_batch})
    duration = time.time() - start_time
    
    if step % 10 == 0:
        examples_per_sec = batch_size / duration
        sec_per_batch = float(duration)
        
        format_str = ('step %d, loss=%.2f (%.1f examples/sec; %.3f sec/batch)')
        print(format_str % (step, loss_value, examples_per_sec, sec_per_batch))

step 0, loss=4.67 (29.7 examples/sec; 4.305 sec/batch)
step 10, loss=3.62 (189.3 examples/sec; 0.676 sec/batch)
step 20, loss=3.15 (158.5 examples/sec; 0.808 sec/batch)
step 30, loss=2.68 (209.3 examples/sec; 0.612 sec/batch)
step 40, loss=2.62 (230.6 examples/sec; 0.555 sec/batch)
step 50, loss=2.33 (262.4 examples/sec; 0.488 sec/batch)
step 60, loss=2.24 (172.4 examples/sec; 0.743 sec/batch)
step 70, loss=2.19 (209.9 examples/sec; 0.610 sec/batch)
step 80, loss=2.06 (203.3 examples/sec; 0.630 sec/batch)
step 90, loss=2.03 (250.0 examples/sec; 0.512 sec/batch)
step 100, loss=1.90 (232.4 examples/sec; 0.551 sec/batch)
step 110, loss=1.88 (184.8 examples/sec; 0.693 sec/batch)
step 120, loss=2.04 (241.4 examples/sec; 0.530 sec/batch)
step 130, loss=1.89 (229.4 examples/sec; 0.558 sec/batch)
step 140, loss=1.81 (225.2 examples/sec; 0.568 sec/batch)
step 150, loss=1.93 (222.9 examples/sec; 0.574 sec/batch)
step 160, loss=1.94 (228.8 examples/sec; 0.560 sec/batch)
step 170, loss=1.77 (273.1

step 1410, loss=1.06 (259.0 examples/sec; 0.494 sec/batch)
step 1420, loss=1.08 (266.8 examples/sec; 0.480 sec/batch)
step 1430, loss=1.33 (205.2 examples/sec; 0.624 sec/batch)
step 1440, loss=1.18 (197.1 examples/sec; 0.649 sec/batch)
step 1450, loss=1.04 (198.6 examples/sec; 0.645 sec/batch)
step 1460, loss=1.30 (236.5 examples/sec; 0.541 sec/batch)
step 1470, loss=1.29 (264.3 examples/sec; 0.484 sec/batch)
step 1480, loss=1.28 (258.9 examples/sec; 0.494 sec/batch)
step 1490, loss=1.31 (191.3 examples/sec; 0.669 sec/batch)
step 1500, loss=1.29 (244.5 examples/sec; 0.523 sec/batch)
step 1510, loss=1.11 (167.5 examples/sec; 0.764 sec/batch)
step 1520, loss=1.24 (201.6 examples/sec; 0.635 sec/batch)
step 1530, loss=1.12 (136.5 examples/sec; 0.938 sec/batch)
step 1540, loss=1.23 (231.4 examples/sec; 0.553 sec/batch)
step 1550, loss=1.24 (267.9 examples/sec; 0.478 sec/batch)
step 1560, loss=1.05 (310.6 examples/sec; 0.412 sec/batch)
step 1570, loss=1.22 (312.0 examples/sec; 0.410 sec/batc

step 2800, loss=1.07 (290.2 examples/sec; 0.441 sec/batch)
step 2810, loss=1.17 (275.4 examples/sec; 0.465 sec/batch)
step 2820, loss=1.07 (275.4 examples/sec; 0.465 sec/batch)
step 2830, loss=1.15 (272.2 examples/sec; 0.470 sec/batch)
step 2840, loss=0.92 (259.2 examples/sec; 0.494 sec/batch)
step 2850, loss=1.39 (283.1 examples/sec; 0.452 sec/batch)
step 2860, loss=1.05 (243.3 examples/sec; 0.526 sec/batch)
step 2870, loss=0.88 (271.0 examples/sec; 0.472 sec/batch)
step 2880, loss=1.02 (254.1 examples/sec; 0.504 sec/batch)
step 2890, loss=1.14 (281.2 examples/sec; 0.455 sec/batch)
step 2900, loss=1.11 (274.6 examples/sec; 0.466 sec/batch)
step 2910, loss=0.84 (286.7 examples/sec; 0.446 sec/batch)
step 2920, loss=1.06 (271.9 examples/sec; 0.471 sec/batch)
step 2930, loss=0.90 (265.7 examples/sec; 0.482 sec/batch)
step 2940, loss=0.85 (275.3 examples/sec; 0.465 sec/batch)
step 2950, loss=0.94 (281.4 examples/sec; 0.455 sec/batch)
step 2960, loss=1.06 (274.3 examples/sec; 0.467 sec/batc

In [19]:
num_examples = 10000
import math
num_iter = int(math.ceil(num_examples / batch_size))
true_count = 0
total_sample_count = num_iter * batch_size
step = 0
test_start_time = time.time()
while step < num_iter:
    image_batch, label_batch = session.run([images_test, labels_test])
    predictions = session.run([top_k_op], feed_dict={image_holder: image_batch, label_holder: label_batch})
    true_count += np.sum(predictions)
    step += 1
end = time.time()
print('测试 ' + str(total_sample_count) + ' 个examples用时为:' + str(end - test_start_time) + '秒')

测试 9984 个examples用时为:9.00916194916秒


In [20]:
precision = true_count * 1.0 / total_sample_count
print('precision @ 1 = %.3f' % precision)

precision @ 1 = 0.713
